###Imports and Setup

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import shutil
import os
from google.colab import drive
drive.mount('/content/drive')
!pip install gurobipy

In [ ]:
!git pull

In [ ]:
import json
from urllib.request import urlopen
from datetime import datetime
import json
import datetime as dt
import matplotlib.pyplot as plt
import os
import math
from copy import copy
import random
pd.set_option('display.max_columns', None)
from itertools import takewhile
import gurobipy as gp
from gurobipy import *
from itertools import product
import timeit
from google.colab import userdata
from gurobipy import Env
import random
!pip install pulp

from dotenv import load_dotenv
load_dotenv()
MATRIX_FILE_PATH = os.getenv('MATRIX_FILE_PATH')
SIZES_FILE_PATH = os.getenv('SIZES_FILE_PATH')

### Helper Functions and Constants

In [ ]:
# Import coenrollment matrix
co = pd.read_csv(MATRIX_FILE_PATH, index_col=0)

# Import number of students taking each exam
e_counts = pd.read_csv(SIZES_FILE_PATH, index_col=0)
# Exam ids are indices of co-enrollment matrix
exams = list(co.index)

# Define available timeslots
timeslots = list(range(1, 25)) # timeslots 1 through 24

# Define penalty weights
CONFLICT_PENALTY = 100
B2B_PENALTY = 1

# Function to initialize random starting solution
def random_initial_solution():
    """Assigns each room to a random timeslot"""
    solution = {}
    for exam in exams:
        solution[exam] = random.choice(timeslots)
    return solution

# Function to initialize a good starting solution
def good_initial_solution():
  solution = {}

  # Sort exams by descending counts of students taking exam
  def exam_count(id):
    return e_counts.loc[id, "size"]
  sorted_exams = sorted(exams, key=exam_count, reverse=True)

  # Traverse through sorted list, adding each exam to the timeslot that currently has the least coenrollments with it
  for e in sorted_exams:
    best_slot = None
    min_conflicts = float('inf')
    for slot in timeslots:
      conflicts = sum([co.loc[e, other_e] for other_e in exams if other_e in solution.keys() and solution[other_e]==slot])
      if conflicts < min_conflicts:
        min_conflicts = conflicts
        best_slot = slot
    solution[e] = best_slot

  return solution

# Compute the cost of a solution
def solution_cost(solution):
    """
    Computes the cost of a timetable.

    For every pair of exams (exam_a, exam_b) that share students (co-enrollment > 0):
      - If they are scheduled in the same timeslot, add CONFLICT_PENALTY.
      - If they are scheduled in consecutive timeslots (difference of 1), add BACK_TO_BACK_PENALTY.

    A lower cost means a better timetable.
    """
    cost = 0
    exam_list = list(solution.keys())
    n = len(exam_list)
    for i in range(n):
        for j in range(i+1, n):
            exam_a = exam_list[i]
            exam_b = exam_list[j]
            # Get the co-enrollment value (number of common students)
            common = co.loc[exam_a, exam_b]
            if common > 0:
                if solution[exam_a] == solution[exam_b]:
                    cost += common* CONFLICT_PENALTY
                elif abs(solution[exam_a] - solution[exam_b]) == 1:
                    cost += common* B2B_PENALTY
    return cost

def solution_cost2(solution):
  """ Computes the cost of a timetable.
  More detailed version that comes from the actual code used.
  """
  results = dict()

  return (100 * results['conflicts'] +
          100 * results['quints'] +
          50 * results['quads'] +
          30 * results['four in five slots'] +
          10 * results['triple in 24h (no gaps)'] +
          10 * results['triple in same day (no gaps)'] +
          2 * results['other b2b'] +
          2 * results['evening/morning b2b'] +
          results['two in three slots'])

# Find a neighbor to an solution
def exam_neighbor(solution):
    """
    Generates a neighbor solution by randomly selecting an exam and assigning
    it a different random timeslot.
    """
    neighbor = solution.copy()
    exam = random.choice(exams)
    available_slots = [ts for ts in timeslots if ts != neighbor[exam]]
    neighbor[exam] = random.choice(available_slots)
    return neighbor

Algorithm-Specific Helper Functions & Constants:

In [ ]:
import math
# Starting threshold
q_max = 10

# Threshold decreasing rate
r = 10**(-2)

# Number of iterations to execute per threshold
its = 5

# Ending threshold
q_min = 1

# Cooling schedule
def threshold_update(curr):
  return curr * math.exp(-r)

print(threshold_update(q_max))

### Algorithm Implementation

In [ ]:
# Implement TA
thresholds = []
def runreg():
  prev_bin = None
  curr_bin = {key:0 for key in exams}
  s = good_initial_solution()
  q = q_max
  while q > q_min:
    for i in range(its):
      s1 = exam_neighbor(s)
      exam_to_move = random.choice(list(s1.keys()))
      if prev_bin is None or prev_bin[exam_to_move] > 0:
        e = solution_cost(s1) - solution_cost(s)
        thresholds.append(int(e))
        if e <= q:
          s = s1
          curr_bin[exam_to_move] = curr_bin[exam_to_move] + 1
          #print(solution_cost(s))
    q = threshold_update(q)
runreg()

In [ ]:
thresholds.sort()
print(thresholds)
bin_size = len(thresholds) // 10
bin_boundaries = []
pos = 0
for i in range(9):
  pos += bin_size
  bin_boundaries.append((thresholds[pos] + thresholds[pos + 1])/2)
print(pos)
print(bin_boundaries)

In [ ]:
bins = bin_boundaries
bin_boundaries = []
for i in bins:
  bin_boundaries.append(float(i))
print(bin_boundaries)
# [-3.0, 0.0, 2.0, 97.0, 102.0, 289.0, 404.5, 772.5, 1766.5]

In [ ]:
bin_boundaries = [0, 1, 2]


In [ ]:
# Implement FastTA
def get_bin_index(q):
    for i in range(len(bin_boundaries) - 1):
        if bin_boundaries[i] <= q < bin_boundaries[i + 1]:
            return i
    return len(bin_boundaries) - 2

def run():
  prev_bin = None
  curr_bin = {key:0 for key in exams}
  s = good_initial_solution()
  q = q_max
  prev_bin_index = get_bin_index(q)
  while q > q_min:
    for i in range(its):
      s1 = exam_neighbor(s)
      exam_to_move = random.choice(list(s1.keys()))
      if prev_bin is None or prev_bin[exam_to_move] > 0:
        e = solution_cost(s1) - solution_cost(s)
        if e <= q:
          print(solution_cost(s1))
          s = s1
          curr_bin[exam_to_move] = curr_bin[exam_to_move] + 1
    q = threshold_update(q)
    # Update bin
    # If q is in the next bin, make prev bin current bin and set current bin to 0
    new_bin_index = get_bin_index(q)
    if new_bin_index > prev_bin_index:
      prev_bin = curr_bin
      curr_bin = {key: 0 for key in exams}
      prev_bin_index = new_bin_index
run()